In [ ]:
!pip install transformers

In [ ]:
!pip install sentencepiece

In [ ]:
#From https://huggingface.co/cardiffnlp/twitter-roberta-base-sentiment
from transformers import AutoModelForSequenceClassification
from transformers import TFAutoModelForSequenceClassification
from transformers import AutoTokenizer
import numpy as np
from scipy.special import softmax
import csv
import urllib.request

# Preprocess text (username and link placeholders)
def preprocess(text):
    new_text = []


    for t in text.split(" "):
        t = '@user' if t.startswith('@') and len(t) > 1 else t
        t = 'http' if t.startswith('http') else t
        new_text.append(t)
    return " ".join(new_text)

# Tasks:
# emoji, emotion, hate, irony, offensive, sentiment
# stance/abortion, stance/atheism, stance/climate, stance/feminist, stance/hillary

task='sentiment'
MODEL = f"cardiffnlp/twitter-roberta-base-{task}"

tokenizer = AutoTokenizer.from_pretrained(MODEL)

# download label mapping
labels=[]
mapping_link = f"https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/{task}/mapping.txt"
with urllib.request.urlopen(mapping_link) as f:
    html = f.read().decode('utf-8').split("\n")
    csvreader = csv.reader(html, delimiter='\t')
labels = [row[1] for row in csvreader if len(row) > 1]

# PT
model = AutoModelForSequenceClassification.from_pretrained(MODEL)
model.save_pretrained(MODEL)

text = "Good night 😊"
text = preprocess(text)
encoded_input = tokenizer(text, return_tensors='pt')
output = model(**encoded_input)
scores = output[0][0].detach().numpy()
scores = softmax(scores)

# # TF
# model = TFAutoModelForSequenceClassification.from_pretrained(MODEL)
# model.save_pretrained(MODEL)

# text = "Good night 😊"
# encoded_input = tokenizer(text, return_tensors='tf')
# output = model(encoded_input)
# scores = output[0][0].numpy()
# scores = softmax(scores)

ranking = np.argsort(scores)
ranking = ranking[::-1]
for i in range(scores.shape[0]):
    l = labels[ranking[i]]
    s = scores[ranking[i]]
    print(f"{i+1}) {l} {np.round(float(s), 4)}")

In [ ]:
url = 'https://raw.githubusercontent.com/trexbatman/projects/master/valid_clean.csv'

In [ ]:
valid_clean = pd.read_csv(url,encoding="ISO-8859-1", low_memory=False)

In [ ]:
valid_clean

0        delivery slots available March April Harrow ar...
1        Wells Fargo committed helping customers experi...
2        What hell going country, shops selling sanitiz...
3        / &amp; Supermarket stockers cashiers. Pharmac...
4        Our friends conducting ongoing research consum...
                               ...                        
10285    Observations local supermarket - majority folk...
10286    Going store barely able stock food unable find...
10287    please anyone hand sanitizer, soap &amp; toile...
10288    Which global recession? V U L I? For I global ...
10289    Those us work companies would prefer stay safe...
Name: OriginalTweet, Length: 10290, dtype: object

In [ ]:
resultsvalid_clean = valid_clean['OriginalTweet'].copy()

In [ ]:
valid_clean

0        delivery slots available March April Harrow ar...
1        Wells Fargo committed helping customers experi...
2        What hell going country, shops selling sanitiz...
3        / &amp; Supermarket stockers cashiers. Pharmac...
4        Our friends conducting ongoing research consum...
                               ...                        
10285    Observations local supermarket - majority folk...
10286    Going store barely able stock food unable find...
10287    please anyone hand sanitizer, soap &amp; toile...
10288    Which global recession? V U L I? For I global ...
10289    Those us work companies would prefer stay safe...
Name: OriginalTweet, Length: 10290, dtype: object

In [ ]:
%%time
encoded_input = tokenizer(valid_clean[0], return_tensors='pt')
output = model(**encoded_input)
scores = output[0][0].detach().numpy()
results = softmax(scores)
for i in range(1,1000): 
  encoded_input = tokenizer(valid_clean[i], return_tensors='pt')
  output = model(**encoded_input)
  scores = output[0][0].detach().numpy()
  scores = softmax(scores)
  results = np.append(results, scores)

CPU times: user 2min 44s, sys: 1.06 s, total: 2min 45s
Wall time: 2min 45s


In [ ]:
results.shape

(3000,)

In [ ]:
np.savetxt("results_part1.csv", results, delimiter=",")

Thank you!